In [ ]:
%load_ext autoreload
%autoreload 2

mapping = {
    "SPARC-10-M":"EEG 1", 
    "SPARC-11-M":"EEG 1",
    "SPARC-12-M":"EEG 1",
    "SPARC-13-M":"EEG 1",
    "SPARC-14-M":"EEG 1",
    "SPARC-15-M":"EEG 1",
    "SPARC-17-M":"EEG 2",
    "SPARC-18-M":"EEG 1",
    "SPARC-2-F":"EEG 1",
    "SPARC-4-F":"EEG 1",
    "SPARC-5-F":"Channel-2",
    "SPARC-8-F":"Channel-10",
    "SPARC-9-F":"Channel-2",
    "SPARC-10-F":"Channel-50",
    "SPARC-11-F":"Channel-18",
    "SPARC-12-F":"Channel-42",
    "SPARC-13-F":"EEG 1",
    "SPARC-14-F":"EEG 1",
    "SPARC-15-F":"EEG 1",
    "SPARC-16-F":"EEG 2"
}

In [ ]:
from lib.utils import *
from lib.models import *
from lib.ekyn import *
from lib.env import *
from lib.datasets import *

class FeatureSet(Dataset):
    def __init__(self,X):
        self.len = len(X)
        self.X = cat([zeros(4,5000),X,zeros(4,5000)])

    def __len__(self):
        return self.len

    def __getitem__(self, idx):
        return self.X[idx:idx+9].flatten()

In [ ]:
for id in get_ekyn_ids():
    X,y = load_eeg_label_pairs(ids=[id])
    print(X.std())

In [ ]:
import os
files = os.listdir(f'{DATA_PATH}/icv/edf')
print(files)
# file = 'SPARC-15-F'
for file in files:
    file = file.replace('.edf','')
    print(file,mapping[file])
    raw = read_raw_edf(f'{DATA_PATH}/icv/edf/{file}.edf',verbose=False)
    eeg = raw.get_data(picks=mapping[file])
    print(eeg.mean(),eeg.std())

In [ ]:
5e-5
5e-2
# .00005
# .05

In [ ]:
import os
files = os.listdir(f'{DATA_PATH}/icv/edf')
print(files)
# file = files[0]
file = 'SPARC-12-F'
file = file.replace('.edf','')
print(file,mapping[file])
raw = read_raw_edf(f'{DATA_PATH}/icv/edf/{file}.edf',verbose=False)
raw.ch_names
eeg = raw.get_data(picks=mapping[file])
print(eeg.mean(),eeg.std())
plt.plot(eeg.reshape(-1,5000)[:10].flatten())

In [ ]:
eeg = eeg/1000

In [ ]:
print(eeg.mean(),eeg.std())


In [ ]:
eeg = eeg.reshape(-1,5000)
eeg = torch.from_numpy(eeg).float()
print(eeg.shape)
dataloader = DataLoader(dataset=FeatureSet(eeg),batch_size=32,shuffle=False)

In [ ]:
def score(dataloader,model,DEVICE=DEVICE):
    with torch.no_grad():
        y_pred = torch.Tensor()
        y_logits = torch.Tensor()
        for Xi in tqdm(dataloader):
            Xi = Xi.to(DEVICE)
            logits = model(Xi)
            y_logits = torch.cat([y_logits,torch.softmax(logits,dim=1).detach().cpu()])
            y_pred = torch.cat([y_pred,torch.softmax(logits,dim=1).argmax(axis=1).detach().cpu()])
    return y_pred,y_logits

In [ ]:
# for i in range(2,16):
i = 0
model = Gandalf()
model.load_state_dict(torch.load(f=f'../models/gandalfs/gandalf_{i}/best_model.pt',map_location='cpu'))
model.to(DEVICE);
y_pred,y_logits = score(dataloader,model)

In [50]:

print(torch.bincount(y_pred.long())/len(y_pred))

tensor([0.0723, 0.3832, 0.5446])


In [ ]:
plot_eeg_and_labels(eeg,torch.ones(len(eeg)),start=0,duration=300)

In [ ]:
path = f'../data/ICV/csv'
import os
for file in os.listdir(f'{path}'):
    df = pd.read_csv(f'{path}/{file}')
    print(df['0'].value_counts(normalize=True))